# Join data files

In [4]:
import pandas as pd

# List of CSV file paths
csv_files = ['ia-data_06-23_08-23.csv', 'ia-data_09-23_12-23.csv', 'ia-data_01-24_03-24.csv', 'ia-data_04-24_06-24.csv', 'ia-data_07-24_11-24.csv']

# Output file
output_file = 'raw_ia_data.csv'

# Create the output file with the header from the first CSV
with open(output_file, 'w') as f_out:
    pd.read_csv(csv_files[0], skip_blank_lines=True).to_csv(f_out, index=False, lineterminator='\n')

# Append the remaining files without the header
for file in csv_files[1:]:
    with open(output_file, 'a') as f_out:
        pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')

print("CSV files appended successfully into 'merged.csv'")


C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(csv_files[0], skip_blank_lines=True).to_csv(f_out, index=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, sk

CSV files appended successfully into 'merged.csv'


In [1]:
import pandas as pd
df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['deviceId', 'Time'], inplace=True)
df.to_csv('raw_ia_data.csv', index=False)

C:\Users\Dario\AppData\Local\Temp\ipykernel_12560\3081564111.py:2: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')


# Get packets previous to a failure

In [6]:
import pandas as pd
from tqdm import tqdm

ALARM_PUMP = 3
ALARM_EQUIPMENT = 5
INTERVAL_HOURS = 8
PACKETS_PER_HOUR = 4
MAX_PACKETS = PACKETS_PER_HOUR * INTERVAL_HOURS
MIN_PACKETS = MAX_PACKETS - 1 * INTERVAL_HOURS

df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['deviceId', 'Time'], inplace=True)
print('START', df.shape[0])

rows_to_drop = []  # Indices to drop
general = {
    'times_to_fail': [], # Time to append
    'interval_groups': [] # Groups to append
}
current = {
    'times_to_fail': [], # Time to append
    'interval_groups': [], # Groups to append,
    'current_rows': []
}
last_row = None
current_group = 1
device_group = ''
for index in tqdm(df.iloc[::-1].index):  # Reverse order
    current_state = df.loc[index, 'state']
    current_time = df.loc[index, 'Time']
    current_device = df.loc[index, 'deviceId']

    if current_state == ALARM_PUMP or current_state == ALARM_EQUIPMENT:    # Apply condition
        if last_row is not None:
            rows_to_drop += current['current_rows'] # Remove invalid interval rows
        current['times_to_fail'] = [0]
        current['interval_groups'] = [current_group]
        current['current_rows'] = [index]
        device_group = current_device
        last_row = {'Time': current_time, 'State': current_state, 'Index': index}
    elif last_row is not None:
        time_diff = last_row['Time'] - current_time
        if  time_diff.total_seconds() > INTERVAL_HOURS * 3600 or current_device != device_group:   # More than 6 hours
            # End interval
            last_row = None
            rows_to_drop.append(index)  # Mark the row for deletion
            if len(current['current_rows']) < MIN_PACKETS or len(current['current_rows']) > MAX_PACKETS:
                rows_to_drop += current['current_rows']
            else:
                general['times_to_fail'] = current['times_to_fail'] + general['times_to_fail']
                general['interval_groups'] = current['interval_groups'] + general['interval_groups']
                current_group += 1

            current = {
                'times_to_fail': [],
                'interval_groups': [],
                'current_rows': []
            }
        else:
            # Append to interval
            current['times_to_fail'].insert(0, time_diff.total_seconds())
            current['interval_groups'].insert(0, current_group)
            current['current_rows'].insert(0, index)
    else:
      rows_to_drop.append(index)  # Mark the row for deletion

# Drop rows in a single operation (more efficient)
df.drop(rows_to_drop, inplace=True)
# Reset the index after dropping rows (optional)
df.reset_index(drop=True, inplace=True)
df['timeToFail'] = general['times_to_fail']
df['intervalGroup'] = general['interval_groups']

df.to_csv('ia_data_events.csv', index=False)

df.describe()

C:\Users\Dario\AppData\Local\Temp\ipykernel_12560\1448317971.py:11: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')


START 3162739


100%|██████████| 3162739/3162739 [01:57<00:00, 26947.24it/s]


,Time,angle,distanceSpeed,flow,latitude,longitude,pressure,rawSpeed,speed,state,timeToFail,intervalGroup
count,57515,57515.000000,57515.000000,57515.000000,57515.000000,57515.000000,57515.000000,57515.000000,57515.000000,57515.000000,57515.000000,57515.000000
mean,2024-01-28 21:50:11.040632832,175.200134,31.247752,194.221607,-33.766039,-63.354381,1.464717,0.014312,0.122750,5.488985,14116.099174,1029.267043
min,2023-06-02 22:01:51,0.000000,-0.260000,43.200000,-40.740237,-112.048533,0.000000,-10.000000,0.000000,0.000000,0.000000,1.000000
25%,2023-11-25 22:38:05,87.250000,0.303000,105.000000,-37.788625,-65.679715,0.600000,-0.078000,0.028700,6.000000,6978.500000,516.000000
50%,2024-01-18 11:41:12,175.000000,0.700000,161.000000,-34.520669,-62.451458,1.400000,0.005860,0.093100,6.000000,14156.000000,1030.000000
75%,2024-02-19 20:49:45,261.000000,1.190000,241.000000,-32.006788,-60.156389,2.100000,0.103000,0.166000,6.000000,21431.500000,1543.000000
max,2024-11-20 09:50:38,360.000000,488031.000000,700.000000,31.029550,-0.087667,10.000000,8.580000,10.000000,8.000000,28800.000000,2058.000000
std,NaN,102.744239,3224.072467,102.179023,6.078084,4.884550,1.045632,0.196858,0.154498,1.474628,8463.753569,594.276951


# Get random intervals with NO failures

In [7]:
import numpy as np
from tqdm import tqdm

CASES_REQUIRED = 2058
ALARM_PUMP = 3
ALARM_EQUIPMENT = 5

df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])

selected_rows = []
interval_groups = []
intervals_num = 1
pbar = tqdm(total=CASES_REQUIRED)
while intervals_num <= CASES_REQUIRED:
  rand_ind = np.random.randint(0, df.shape[0] - 30) # Padding of 30 last lines
  init_time = df.loc[rand_ind, 'Time']
  init_device = df.loc[rand_ind, 'deviceId']

  curr_ind = rand_ind
  go_on = True
  current_rows = []
  while go_on: # Fix
    row = df.loc[curr_ind]
    time_diff = row['Time'] - init_time

    if row['state'] == ALARM_PUMP or row['state'] == ALARM_EQUIPMENT or row['deviceId'] != init_device:
      # Do not keep the interval
      go_on = False 
    elif time_diff.total_seconds() > INTERVAL_HOURS * 3600:
      # Keep interval
      if len(current_rows) > MIN_PACKETS:
        selected_rows += current_rows
        interval_groups += [intervals_num] * len(current_rows)
        intervals_num += 1
        pbar.update(1)
      go_on = False
    else:
      current_rows.append(row)
      curr_ind += 1

pbar.close()

new_df = pd.DataFrame(selected_rows, columns=df.columns)
new_df['timeToFail'] = [-1] * new_df.shape[0]
new_df['intervalGroup'] = interval_groups
new_df.to_csv('ia_data_no_fails.csv', index=False)
    



C:\Users\Dario\AppData\Local\Temp\ipykernel_12560\3140033606.py:8: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')
100%|██████████| 2058/2058 [00:08<00:00, 239.09it/s]
